## Tool Calling or Function Calling
- LLM Automatically calls the function based on the query
- Function parameters are automatically passed to the function
- It is one of the essential requirements of the Agent
- Not all LLM supports tool calling.

https://python.langchain.com/docs/concepts/tools/

In [27]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_groq import ChatGroq

llm = ChatOllama(model='llama3.2', base_url='http://localhost:11434')
llm.invoke('hi')

groq_llm = ChatGroq(model="llama-3.2-3b-preview")

### Custom Tools

https://python.langchain.com/docs/how_to/custom_tools/

In [29]:
from langchain_core.tools import tool

In [54]:
@tool
def add(x: int, y: int) -> int:
    """
    This function adds two integers.
    Args:
        x: First integer
        y: Second integer
    Returns:
        Sum of x and y
    """
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """
    This function multiplies two numbers.
    Args:
        x: First number
        y: Second number
    Returns:
        Product of x and y
    """
    return x * y

In [46]:
# @tool
# def multiply(x: float, y: float) -> float:
#     """Multiply two numbers."""
#     return x * y

# @tool
# def add(x: int, y: int) -> int:
#     """Add two numbers."""
#     return x + y

# @tool 
# def divide(x:float, y:float) -> float:
#     """Divide two numbers."""
#     return x / y

# tools = [multiply, add, divide]

In [55]:
type(add)

langchain_core.tools.structured.StructuredTool

In [56]:
add.name,add.description,add.args,add.args_schema,add.args_schema.schema()

('add',
 'This function adds two integers.\nArgs:\n    x: First integer\n    y: Second integer\nReturns:\n    Sum of x and y',
 {'x': {'title': 'X', 'type': 'integer'},
  'y': {'title': 'Y', 'type': 'integer'}},
 langchain_core.utils.pydantic.add,
 {'description': 'This function adds two integers.\nArgs:\n    x: First integer\n    y: Second integer\nReturns:\n    Sum of x and y',
  'properties': {'x': {'title': 'X', 'type': 'integer'},
   'y': {'title': 'Y', 'type': 'integer'}},
  'required': ['x', 'y'],
  'title': 'add',
  'type': 'object'})

In [57]:
add.invoke({'x':4,'y':5})

9

In [50]:
multiply.invoke({'x':4,'y':5})

20.0

In [71]:
tools = [add,multiply]
llm_with_tools = groq_llm.bind_tools(tools)

In [86]:
question  = "what is 1 plus 2?"
response = llm_with_tools.invoke(question)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ee3z', 'function': {'arguments': '{}', 'name': 'add{x=1,y=2}'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 379, 'total_tokens': 392, 'completion_time': 0.008144263, 'prompt_time': 0.061244108, 'queue_time': 0.017670740999999997, 'total_time': 0.069388371}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-14b97b65-0be3-46c0-9533-4ce3f290f9db-0', tool_calls=[{'name': 'add{x=1,y=2}', 'args': {}, 'id': 'call_ee3z', 'type': 'tool_call'}], usage_metadata={'input_tokens': 379, 'output_tokens': 13, 'total_tokens': 392})

In [87]:
print(response.content)

In [88]:
question  = "what is 1 multiplied by 2, also what is 11 plus 22?"
response = llm_with_tools.invoke(question)
response


AIMessage(content='<multiply=x=1,y=2>{"x": 1}</multiply>\n<add>x=1,y=22{"x": 11}</add>\n<multiply=x=2,y=2>{"y": 2}</multiply>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 389, 'total_tokens': 438, 'completion_time': 0.030592929, 'prompt_time': 0.062735028, 'queue_time': 0.018365331, 'total_time': 0.093327957}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb5bde5a-ae9b-4962-90ce-11e5a43c256f-0', usage_metadata={'input_tokens': 389, 'output_tokens': 49, 'total_tokens': 438})

In [83]:
print(response.content)

<multiply=x=1,y=2>{multiply} {x} {y} = 2</multiply>
<add>x=11,y=22">add {x} {y} = 33</add>


In [90]:
question  = "what is 1 multiplied by 2?"
response = llm_with_tools.invoke(question)
response

AIMessage(content='<multiply=x=1>y=2></multiply>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 759, 'total_tokens': 771, 'completion_time': 0.007512461, 'prompt_time': 0.119959229, 'queue_time': -0.133850038, 'total_time': 0.12747169}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4755cf3-c5fd-48cd-bd27-3efac9f1b5e4-0', usage_metadata={'input_tokens': 759, 'output_tokens': 12, 'total_tokens': 771})

In [91]:
print(response.content)

<multiply=x=1>y=2></multiply>


### create an async implementation, like this:

In [92]:
from langchain_core.tools import tool


@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

## How to create async tools

LangChain Tools implement the Runnable interface 🏃.

- All Runnables expose the invoke and ainvoke methods (as well as other methods like batch, abatch, astream etc).

- So even if you only provide an sync implementation of a tool, you could still use the ainvoke interface, but there are some important things to know:

- LangChain's by default provides an async implementation that assumes that the function is expensive to compute, so it'll delegate execution to another thread.
If you're working in an async codebase, you should create async tools rather than sync tools, to avoid incuring a small overhead due to that thread.

- If you need both sync and async implementations, use StructuredTool.from_function or sub-class from BaseTool.

- If implementing both sync and async, and the sync code is fast to run, override the default LangChain async implementation and simply call the sync code.
You CANNOT and SHOULD NOT use the sync invoke with an async tool.

In [93]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(
    await calculator.ainvoke({"a": 2, "b": 5})
)  # Uses default LangChain async implementation incurs small overhead

6
10


##  Calling In-Built Tool

## Different Types of Searching Tools

## DuckDuckGo Search

https://python.langchain.com/docs/integrations/tools/ddg/

In [69]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Obama's first name?")

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed ... Barack Obama, the 44th President of the United States, broke barriers as the first African-American president and implemented significant healthcare reforms during his tenure. ... Secret Service Code Names of 11 U.S. Presidents. Barack Obama\'s Presidential Legacy. ... President Barack Obama\'s First Words to America" (2009) "The Audacity of ... Barack Hussein Obama II (US i / b ə ˈ r ɑː k h uː ˈ s eɪ n ɵ ˈ b ɑː m ə /, born August 4, 1961) is the 44th and current President of the United States, and the first African American to hold the office. Born in Honolulu, Hawaii, Obama is a graduate of Colu

### To get more additional information (e.g. link, source) use DuckDuckGoSearchResults()

In [41]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.invoke("Obama")

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'snippet: A comprehensive overview of the 44th president of the United States, his achievements, policies, and controversies. Learn about his election, inauguration, administration, judicial appointments, domestic and foreign affairs, and legacy., title: Presidency of Barack Obama - Wikipedia, link: https://en.wikipedia.org/wiki/Presidency_of_Barack_Obama, snippet: Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009–17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005–08). He was the third African American to be elected to that body since the end of Reconstruction (1877)., title: Barack Obama | Biography, Parents, Education, Presidency, Books ..., link: https://www.britannica.com/biography/Barack-Obama, snippet: Learn about the life, education and achievements of Barack Obama, the 44th and first African American president of the United States. Exp

By default the results are returned as a comma-separated string of key-value pairs from the original search results. You can also choose to return the search results as a list by setting output_format="list" or as a JSON string by setting output_format="json"

In [42]:
search = DuckDuckGoSearchResults(output_format="json")

search.invoke("Obama")

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'[{"snippet": "A comprehensive overview of the 44th president of the United States, his achievements, policies, and controversies. Learn about his election, inauguration, administration, judicial appointments, domestic and foreign affairs, and legacy.", "title": "Presidency of Barack Obama - Wikipedia", "link": "https://en.wikipedia.org/wiki/Presidency_of_Barack_Obama"}, {"snippet": "Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009\\u201317) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005\\u201308). He was the third African American to be elected to that body since the end of Reconstruction (1877).", "title": "Barack Obama | Biography, Parents, Education, Presidency, Books ...", "link": "https://www.britannica.com/biography/Barack-Obama"}, {"snippet": "Learn about the life, education and achievements of Barack Obama, the 44th and first African

### Tavily Search

In [95]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True
)

In [96]:
question = "what is today's stock market news?"
search.invoke(question)

[{'url': 'https://www.investors.com/news/stock-market-today-stock-market-news/',
  'content': "Today: Stock Market News And Analysis\nLATEST MARKET UPDATES\nCurrent Recommended Market Exposure Level\nSTOCK MARKET TODAY\nWith regular updates throughout each trading session, the Stock Market Today column highlights the latest stock market news and alerts you to any changes in market trend. By tracking the market in terms of three stages\xa0—\xa0market in uptrend, uptrend under pressure and market correction — the Stock Market Today helps you stay on top of the latest stock market news while keeping any latest fluctuations in perspective to help you decide what action, if any, to take.\n Start With This 3-Step Routine\nSee Which Stocks Were Just Added To – And Removed From – IBD Stock Lists\nHow To Buy Stocks\xa0|\xa0Buying Checklist\nWhen To Sell Stocks\xa0|\xa0Selling Checklist\nIBD Newsletters\nIBD Newsletters\nGet these newsletters delivered to your inbox & more info about our product

## Google Serper

In [100]:
from langchain_community.utilities import GoogleSerperAPIWrapper

search = GoogleSerperAPIWrapper()

search.run("Who is Balaji Srinivasan and What Does He Teach Through His Podcast and Newsletter?")

"Balaji Srinivasan goes in-depth on Web3, Crypto, the advantages and disadvantages of established democracies, the Russia-Ukraine situation, housing as an ... Missing: Teach | Show results with:Teach. Balaji S. Srinivasan (born May 24, 1980) is an American entrepreneur and investor. He was the co-founder of Counsyl, the former chief technology officer ... Balaji explains what Kim Kardashian can teach us about referential integrity and authenticity on social media. He uses a framework from Magic ... Our guest today has a PhD in Electrical Engineering from Stanford, founded and sold a genomics company, became an early investor in crypto ... Missing: Newsletter? | Show results with:Newsletter?. My guest today is Balaji Srinivasan, an angel investor and entrepreneur. When it comes to the future, he's the single most creative person I know. Balaji Srinivasan is a technologist and entrepreneur who has become a prominent figure in the digital currency and technology sectors. Born and ... The 

### Obtaining results with metadata

If you would also like to obtain the results in a structured way including metadata. For this we will be using the results method of the wrapper.

In [102]:
search = GoogleSerperAPIWrapper()
results = search.results("Apple Inc.")
print(results)


{'searchParameters': {'q': 'Apple Inc.', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'engine': 'google'}, 'knowledgeGraph': {'title': 'Apple', 'type': 'Technology company', 'website': 'http://www.apple.com/', 'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwGQRv5TjjkycpctY66mOg_e2-npacrmjAb6_jAWhzlzkFE3OTjxyzbA&s=0', 'description': 'Apple Inc. is an American multinational corporation and technology company headquartered and incorporated in Cupertino, California, in Silicon Valley. It is best known for its consumer electronics, software, and services.', 'descriptionSource': 'Wikipedia', 'descriptionLink': 'https://en.wikipedia.org/wiki/Apple_Inc.', 'attributes': {'Customer service': '1 (800) 275-2273', 'Founders': 'Steve Jobs, Steve Wozniak, and Ronald Wayne', 'Headquarters': 'Cupertino, CA', 'CEO': 'Tim Cook (Aug 24, 2011–)', 'CFO': 'Luca Maestri', 'Founded': 'April 1, 1976, Los Altos, CA'}}, 'organic': [{'title': 'Apple', 'link': 'https://www.apple.com/', 

### Searching for Google Places

In [201]:
search = GoogleSerperAPIWrapper(type="places")
results = search.results("Mexico restaurants in chennai")
print(results)

{'searchParameters': {'q': 'Mexico restaurants in chennai', 'gl': 'us', 'hl': 'en', 'type': 'places', 'num': 10, 'engine': 'google'}, 'places': [{'position': 1, 'title': 'Mexe', 'address': '2, 13, West St, Cooperative Colony, Sri Ram Nagar, Alwarpet, Chennai, Tamil Nadu 600018, India', 'latitude': 13.031625900000002, 'longitude': 80.2493866, 'rating': 4.2, 'ratingCount': 934, 'category': 'Mexican restaurant', 'phoneNumber': '+91 88254 35460', 'cid': '10897027122125639200'}, {'position': 2, 'title': 'Taco bowl it', 'address': 'No.25, thamizhar nagar, Shenoy Nagar, Chennai, Tamil Nadu 600030, India', 'latitude': 13.082887399999999, 'longitude': 80.22553119999999, 'rating': 4.7, 'ratingCount': 664, 'category': 'Mexican restaurant', 'cid': '9966710137607899257'}, {'position': 3, 'title': 'Mex It Up - World On A Plate', 'address': 'Mex It Up, 59, KB Dasan Rd, opposite SIET COLLEGE, Seethammal Extension, Teynampet, Chennai, Tamil Nadu 600018, India', 'latitude': 13.035398299999999, 'longitud

### Wikipedia Search

In [111]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())


question = "what is the capital of Tamilnadu?"
question = "What is LLM?"

print(wikipedia.invoke(question))

Page: Large language model
Summary: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.
The largest and most capable LLMs are generative pretrained transformers (GPTs). Modern models can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.

Page: Prompt engineering
Summary: Prompt engineering is the process of structuring an instruction that can be interpreted and understood by a generative artificial intelligence (AI) model. 
A prompt is natural language text describing the task that an AI should perform. A prompt for a text-to-text language model can be a query such

### PubMed

- PubMed® comprises more than 35 million citations for biomedical literature from MEDLINE, life science journals, and online books. Citations may include links to full text content from PubMed Central and publisher web sites.



In [112]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

search = PubmedQueryRun()

print(search.invoke("What is the latest research on COVID-19?"))

No good PubMed Result was found


### Tool Calling with LLM

In [153]:
@tool
def wikipedia_search(query: str) -> str:
    """
    Search Wikipedia for general information. Use this for historical, cultural, or factual queries.

    Args:
        query: The search query.
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.run(query)
    return response

@tool
def pubmed_search(query: str) -> str:
    """
    Search PubMed for medical and life sciences queries. Use this for health, medicine, or biology-related questions.

    Args:
        query: The search query.
    """
    search = PubmedQueryRun()
    response = search.run(query)
    return response

@tool
def google_search_place(query: str) -> str:
    """
    Search Google for information about places. Use this for location-based queries, such as landmarks, cities, or addresses.

    Args:
        query: The search query.
    """
    search = GoogleSerperAPIWrapper()
    response = search.run(query)
    return response

@tool
def tavily_search(query: str) -> str:
    """
    Search the web for real-time and latest information. Use this for news, stock market, weather updates, or other time-sensitive queries.

    Args:
        query: The search query.
    """
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )
    response = search.run(query)
    return response

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integer numbers together. Use this for mathematical calculations.

    Args:
        a: First integer.
        b: Second integer.
    """
    return a * b

@tool
def add(a: int, b: int) -> int:
    """
    Add two integer numbers together. Use this for mathematical calculations.

    Args:
        a: First integer.
        b: Second integer.
    """
    return a + b

In [151]:
from langchain.agents import initialize_agent

# Initialize the language model
llm = groq_llm

# List of tools
tools = [wikipedia_search, pubmed_search, google_search_place, tavily_search]

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)



> Entering new AgentExecutor chain...
Thought: To find the answer to this question, I should search for the capital of France on Wikipedia.

Action: wikipedia_search
Action Input: "capital of France"

Observation: Page: Capital punishment in France
Summary: Capital punishment in France (French: peine de mort en France) is banned by Article 66-1 of the Constitution of the French Republic, voted as a constitutional amendment by the Congress of the French Parliament on 19 February 2007 and simply stating "No one can be sentenced to the death penalty" (French: Nul ne peut être condamné à la peine de mort). The death penalty was already declared illegal on 9 October 1981 when President François Mitterrand signed a law prohibiting the judicial system from using it and commuting the sentences of the seven people on death row to life imprisonment. The last execution took place by guillotine, being the main legal method since the French Revolution; Hamida Djandoubi, a Tunisian citizen convict

In [155]:
# Query 1: General knowledge (should use Wikipedia)
result = agent.run("What is the capital of France?")
print(result)

# Query 2: Medical query (should use PubMed)
result = agent.run("How to treat lung cancer?")
print(result)

# Query 3: Location-based query (should use Google Search)
result = agent.run("Where is the Eiffel Tower located?")
print(result)

# Query 4: Real-time information (should use Tavily)
result = agent.run("What is the current weather in New York?")
print(result)

# Query 5: Mathematical calculation (should use multiply or add)
result = agent.run("What is 5 multiplied by 3?")
print(result)



> Entering new AgentExecutor chain...
Question: What is the capital of France?
Thought: I need to find a reliable source to determine the capital of France.
Action: wikipedia_search
Action Input: "capital of France"
Observation: Page: List of capitals of France
Summary: This is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.



Page: Capital punishment in France
Summary: Capital punishment in France (French: peine de mort en France) is banned by Article 66-1 of the Constitution of the French Republic, voted as a constitutional amendment by the Congress of the French Parliament on 19 February 2007 and simply stating "No one can be sentenced to the death penalty" (French: Nul ne peut être condamné à la peine de mort). The death penalty was already declared illegal on 9 October 1981 when President François Mitterrand signed a law prohibiting the judicial system from using it and commuting the sentences of the seven people on

In [152]:
# Use the agent
result = agent.run("How to treat lung cancer?")
print(result)



> Entering new AgentExecutor chain...
Thought: To determine how to treat lung cancer, I should first look up general information about the treatment options available for lung cancer.

Action: wikipedia_search
Action Input: "lung cancer treatment"
Observation: Page: Lung cancer
Summary: Lung cancer, also known as lung carcinoma, is a malignant tumor that begins in the lung. Lung cancer is caused by genetic damage to the DNA of cells in the airways, often caused by cigarette smoking or inhaling damaging chemicals. Damaged airway cells gain the ability to multiply unchecked, causing the growth of a tumor. Without treatment, tumors spread throughout the lung, damaging lung function. Eventually lung tumors metastasize, spreading to other parts of the body.
Early lung cancer often has no symptoms and can only be detected by medical imaging. As the cancer progresses, most people experience nonspecific respiratory problems: coughing, shortness of breath, or chest pain. Other symptoms depend

In [156]:
@tool
def wikipedia_search(query: str) -> str:
    """
    Search Wikipedia for general information.

    Args:
        query: The search query.
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)  # Use .run() instead of .invoke()
    return response

@tool
def pubmed_search(query: str) -> str:
    """
    Search PubMed for medical and life sciences queries.

    Args:
        query: The search query.
    """
    search = PubmedQueryRun()
    response = search.run(query)  
    return response

@tool
def google_search_place(query: str) -> str:
    """
    Search Google for information about places.

    Args:
        query: The search query.
    """
    search = GoogleSerperAPIWrapper(type="places")  
    response = search.results(query)  
    return response

@tool
def tavily_search(query: str) -> str:
    """
    Search the web for real-time and latest information (e.g., news, stock market, weather updates).

    Args:
        query: The search query.
    """
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )
    response = search.invoke(query)  
    return response

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integer numbers together.

    Args:
        a: First integer.
        b: Second integer.
    """
    return a * b  # No need to convert to int again

@tool
def add(a: int, b: int) -> int:
    """
    Add two integer numbers together.

    Args:
        a: First integer.
        b: Second integer.
    """
    return a + b 

In [181]:
# List of tools
tools = [wikipedia_search, pubmed_search, google_search_place, tavily_search, add, multiply]
list_of_tools = { tool.name: tool for tool in tools }
# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

In [182]:
list_of_tools

{'wikipedia_search': StructuredTool(name='wikipedia_search', description='Search Wikipedia for general information. Use this for historical, cultural, or factual queries.\n\nArgs:\n    query: The search query.', args_schema=<class 'langchain_core.utils.pydantic.wikipedia_search'>, func=<function wikipedia_search at 0x00000132AAF96F20>),
 'pubmed_search': StructuredTool(name='pubmed_search', description='Search PubMed for medical and life sciences queries. Use this for health, medicine, or biology-related questions.\n\nArgs:\n    query: The search query.', args_schema=<class 'langchain_core.utils.pydantic.pubmed_search'>, func=<function pubmed_search at 0x00000132AAF974C0>),
 'google_search_place': StructuredTool(name='google_search_place', description='Search Google for information about places. Use this for location-based queries, such as landmarks, cities, or addresses.\n\nArgs:\n    query: The search query.', args_schema=<class 'langchain_core.utils.pydantic.google_search_place'>, f

In [183]:
# Query 1: General knowledge (should use Wikipedia)
query = "What is the capital of France?"
response = llm_with_tools.invoke(query)
print(response)

# Query 2: Medical query (should use PubMed)
query = "How to treat lung cancer?"
response = llm_with_tools.invoke(query)
print(response)

# Query 3: Location-based query (should use Google Search)
query = "Where is the Eiffel Tower located?"
response = llm_with_tools.invoke(query)
print(response)

# Query 4: Real-time information (should use Tavily)
query = "What is the current weather in New York?"
response = llm_with_tools.invoke(query)
print(response)

# Query 5: Mathematical calculation (should use multiply or add)
query = "What is 5 multiplied by 3?"
response = llm_with_tools.invoke(query)
print(response)

content='<wikipedia_search>query=Capital of France</wikipedia_search>' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 784, 'total_tokens': 799, 'completion_time': 0.009322016, 'prompt_time': 0.123985612, 'queue_time': 0.017873374999999997, 'total_time': 0.133307628}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None} id='run-832a7b1e-2152-4385-b4da-47585c394f55-0' usage_metadata={'input_tokens': 784, 'output_tokens': 15, 'total_tokens': 799}
content='</function=wikipedia_search>{"query":"treatment of lung cancer"}' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 783, 'total_tokens': 798, 'completion_time': 0.009332646, 'prompt_time': 0.124025841, 'queue_time': 0.018143818000000006, 'total_time': 0.133358487}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs

In [184]:
query = "Where is the Eiffel Tower located?"
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gmv5', 'function': {'arguments': '{"query": "Eiffel Tower location"}', 'name': 'wikipedia_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 1571, 'total_tokens': 1591, 'completion_time': 0.012433335, 'prompt_time': 0.245871123, 'queue_time': -0.260088153, 'total_time': 0.258304458}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a4a5c4bd-43cf-4d84-8b1d-e248310e15bd-0', tool_calls=[{'name': 'wikipedia_search', 'args': {'query': 'Eiffel Tower location'}, 'id': 'call_gmv5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1571, 'output_tokens': 20, 'total_tokens': 1591})

In [185]:
query = "What is the current weather in New York?"
llm_with_tools.invoke(query)

AIMessage(content='</function=tavily_search {"query": "New York weather today"}>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 786, 'total_tokens': 803, 'completion_time': 0.01054228, 'prompt_time': 0.125141278, 'queue_time': 0.018927101, 'total_time': 0.135683558}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd8e03c3-7af2-4826-b427-33ed42e615d4-0', usage_metadata={'input_tokens': 786, 'output_tokens': 17, 'total_tokens': 803})

### Generate Final Result with Tool Calling

In [186]:
from langchain_core.messages import HumanMessage,AIMessage

In [187]:
query = "teach me about llm?"

messages = [HumanMessage(query)]

tool_calls = llm_with_tools.invoke(messages)
print(tool_calls)
messages.append(tool_calls)
print(messages)
tool_calls = tool_calls.tool_calls

content='</function=wikipedia_search>{"query": "LLM"}' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 784, 'total_tokens': 797, 'completion_time': 0.008112832, 'prompt_time': 0.12417505, 'queue_time': 0.018401877999999997, 'total_time': 0.132287882}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None} id='run-b9a3cbcc-d844-4015-880d-e7a7e9b3de1f-0' usage_metadata={'input_tokens': 784, 'output_tokens': 13, 'total_tokens': 797}
[HumanMessage(content='teach me about llm?', additional_kwargs={}, response_metadata={}), AIMessage(content='</function=wikipedia_search>{"query": "LLM"}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 784, 'total_tokens': 797, 'completion_time': 0.008112832, 'prompt_time': 0.12417505, 'queue_time': 0.018401877999999997, 'total_time': 0.132287882}, 'model_name': 'llama-3.2-3b-preview', 'system_f

In [188]:
from langchain_core.messages import HumanMessage, AIMessage

# Query
query = "What is medicine for lung cancer?"

# Initialize messages with the user query
messages = [HumanMessage(content=query)]

# Invoke the LLM to decide which tool to use
ai_msg = llm_with_tools.invoke(messages)

# Append the LLM's response to the messages
messages.append(ai_msg)

In [193]:
# Process tool calls (if any)
for tool_call in ai_msg.tool_calls:
    print(tool_call)

    # Get the tool name
    name = tool_call['name'].lower()

    # Select the tool
    selected_tool = list_of_tools[name]

    # Invoke the tool with the provided arguments
    tool_msg = selected_tool.invoke(tool_call['args'])

    # Append the tool's response to the messages
    messages.append(AIMessage(content=tool_msg))

In [194]:
messages

[HumanMessage(content='What is medicine for lung cancer?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='</function=tavily_search>{"query":"medicine for lung cancer"}}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 784, 'total_tokens': 799, 'completion_time': 0.009326886, 'prompt_time': 0.12421368, 'queue_time': 0.020190048000000002, 'total_time': 0.133540566}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None}, id='run-2beb5975-f904-44c7-a696-7de087ba40ba-0', usage_metadata={'input_tokens': 784, 'output_tokens': 15, 'total_tokens': 799})]

In [197]:
response = llm_with_tools.invoke(messages)
print(response)

content='' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 798, 'total_tokens': 799, 'completion_time': 0.000612042, 'prompt_time': 0.125972278, 'queue_time': -9223372036.98075, 'total_time': 0.12658432}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None} id='run-26f6b0e8-74d9-490e-9bad-e12661c10895-0' usage_metadata={'input_tokens': 798, 'output_tokens': 1, 'total_tokens': 799}
